In [4]:
import os
import sys
os.chdir('/home/zongchen/mmd_flow_cubature/')
sys.path.append('/home/zongchen/mmd_flow_cubature/')
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

from mmd_flow.distributions import Distribution
from mmd_flow.kernels import gaussian_kernel
from mmd_flow.mmd import mmd_fixed_target
from mmd_flow.gradient_flow import gradient_flow

In [19]:
d = 2
cov = jnp.eye(d)
# cov = jnp.array([[2.0, 1.0], [1.0, 2.0]])
# means = jnp.array([-2.0, 2.0])
means = jnp.array([-0.0, 0.0])

cov_inv = jnp.linalg.inv(cov)
temp = jnp.exp(0.5 * (means.T @ cov_inv @ jnp.linalg.inv(cov_inv + 2 * jnp.eye(d)) @ cov_inv @ means))
temp *= jnp.exp(-0.5 * means.T @ cov_inv @ means)
temp * jnp.sqrt(jnp.linalg.det(2 * cov + jnp.eye(d)))
cov_new = jnp.linalg.inv(cov_inv + 2 * jnp.eye(d))
integral = temp * jnp.sqrt(jnp.linalg.det(cov_inv)) * jnp.sqrt(jnp.linalg.det(cov_new))
print(integral)

samples = jax.random.multivariate_normal(jax.random.PRNGKey(0), means, cov, (1000000,))
f_ = jnp.exp(-jnp.sum(samples**2, axis=1))
integral = jnp.mean(f_)
print(integral)

0.33333334
0.3335045


0.3301388
